# Programs with arguments

In this document, we will learn how to pass arguments to our program.

Let's create another file with our program [./source_files/program_3.py](./source_files/program_3.py). 

Instead of having the circuit defined inside the program (like we did in first example), we will pass it as an argument. We will also save the results, so we can access them later by calling [save_result](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.save_result.html#quantum_serverless.core.save_result).

Here is the program:

```python
# source_files/program_3.py

from quantum_serverless import get_arguments, save_result

from qiskit.primitives import Sampler

# get all arguments passed to this program
arguments = get_arguments()

# get specific argument that we are interested in
circuit = arguments.get("circuit")

sampler = Sampler()

quasi_dists = sampler.run(circuit).result().quasi_dists

print(f"Quasi distribution: {quasi_dists[0]}")

# saving results of a program
save_result({
    "quasi_dists": quasi_dists[0]
})
```

As you can see, the circuit construction is not inside the program anymore. Instead, we parse the arguments by calling the [get_arguments](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.serializers.get_arguments.html#quantum_serverless.serializers.get_arguments) function.

First, we will create circuit that we want to pass as an argument to the program.

In [ ]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

Now let's create and configure our client

In [ ]:
from quantum_serverless import Program

program = Program(
    title="Program with arguments",
    entrypoint="program_3.py",
    working_dir="./source_files/"
)

In [ ]:
from quantum_serverless import QuantumServerless, Provider
import os

In [ ]:
provider = Provider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

Run the program using the `run` method and by passing an `arguments` parameter in form of a dictionary with the "circuit" string as a key and our `QuantumCircuit` as the value

In [ ]:
job = serverless.run(program, arguments={"circuit": circuit})

Retrieve the results from the client

In [ ]:
job.result()